### Import required modules

In [1]:
import cherrypy
import json
import redis

### Connect to Redis

In [2]:
REDIS_HOST = 'redis-11392.c300.eu-central-1-1.ec2.redns.redis-cloud.com'
REDIS_PORT = 11392
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'nVukRxv3hvJTuYLkK6n4XdWF8etIzoOO'

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)

is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

Redis Connected: True


### Check status

In [3]:
class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

### Sensors Endpoint

In [4]:
class Sensors(object):
    exposed = True

    def POST(self, *path, **query):
        body = cherrypy.request.body.read()
        # print(body)
        body_dict = json.loads(body.decode())
        # print(body_dict)

        mac_address = body_dict.get('mac_address', None)

        if mac_address is None:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request body.')

        try:
            redis_client.ts().create(f'{mac_address}:temperature', retention_msecs=24*60*60*1000)
        except redis.ResponseError:
            raise cherrypy.HTTPError(409, 'Sensor already exists.')

        try:
            redis_client.ts().create(f'{mac_address}:humidity', retention_msecs=24*60*60*1000)
        except redis.ResponseError:
            raise cherrypy.HTTPError(409, 'Sensor already exists.')

        return

### Sensor Endpint 

In [5]:

import datetime
from dateutil.parser import parse as parse_date

class Data(object):
    exposed = True

    
    def GET(self, *path, **query):

        #path paramters: mac_address
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        # Check for required query parameters
        start_date = query.get('start_date')
        end_date = query.get('end_date')

        if not start_date:
            raise cherrypy.HTTPError(400, 'Missing start date in the request parameters.')

        if not end_date:
            raise cherrypy.HTTPError(400, 'Missing end date in the request parameters.')

        # Validate date format
        try:
            start_date_parsed = parse_date(start_date)
        except ValueError:
            raise cherrypy.HTTPError(400, 'Wrong format for start date in the request parameters.')

        try:
            end_date_parsed = parse_date(end_date)
        except ValueError:
            raise cherrypy.HTTPError(400, 'Wrong format for end date in the request parameters.')

        # Check date range validity
        if end_date_parsed <= start_date_parsed:
            raise cherrypy.HTTPError(400, 'End date smaller or equal than start date.')

        # Convert date to timestamps in ms
        start_timestamp = int(start_date_parsed.timestamp() * 1000)
        end_timestamp = int(end_date_parsed.timestamp() * 1000)

        # Attempt to fetch the data from Redis
        try:
            #timestamps = redis_client.ts().range(f'{mac_address}:timestamp', start_timestamp, end_timestamp)
            temperatures = redis_client.ts().range(f'{mac_address}:temperature', start_timestamp, end_timestamp)
            humidities = redis_client.ts().range(f'{mac_address}:humidity', start_timestamp, end_timestamp)
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        # Prepare the response
        response_dict = {
            "mac_address": mac_address , 
            "timestamp": [int(t[0]) for t in temperatures],  # Use temperatures for timestamps
            "temperature": [int(t[1]) for t in temperatures],
            "humidity": [int(h[1]) for h in humidities],
        }

        response = json.dumps(response_dict)
        return response



### Check server status

In [1]:
import requests
import pandas as pd

host = "https://8a9d9526-dc21-42d6-ba37-8f708634743d.deepnoteproject.com"
mac_address = "0xe45f01e89bcc"

#check the server status
response = requests.get(host + '/status')
if response.status_code == 200:
    # print(response.json())
    status = response.json()['status']
    print(f'The server is {status}.')
else:
    print('The server is offline.')
    exit()




The server is online.


### Add sensor node

In [3]:
payload = {'mac_address': '0xdca632c91d8d'}
# Specify the request body with "json" argument of the post method.
response = requests.post(host + '/sensors', json=payload)

# Check the response code. The expected value is 200.
if response.status_code == 200:
    print('Sensor timeseries added.')
else:
    print(response.status_code, response.reason)
    exit()

Sensor timeseries added.


### Retrieve temperature and humidity data

In [9]:
# Make the GET request
base_url = "https://8a9d9526-dc21-42d6-ba37-8f708634743d.deepnoteproject.com/data"
start_date = '2024-12-29'
end_date = '2024-12-30'
response = requests.get(f"{base_url}/{mac_address}", params={"start_date": start_date, "end_date": end_date})

# Check the response
if response.status_code != 200:
    print(f"Failed to retrieve data: {response.status_code} - {response.text}")
else:
    # Parse the response
    data = response.json()
    
    # Create DataFrames
    timestamps = pd.to_datetime(data['timestamp'], unit='ms')  # Convert timestamps from ms to datetime
    humidity_df = pd.DataFrame({'Time': timestamps, 'Humidity': data['humidity']})
    temperature_df = pd.DataFrame({'Time': timestamps, 'Temperature': data['temperature']})

In [15]:
(lambda: DeepnoteChart(humidity_df, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"trail","color":"#2266D3","tooltip":true},"encoding":{"x":{"axis":{"grid":false},"sort":"ascending","type":"temporal","field":"Time","scale":{"type":"linear","zero":false}},"y":{"axis":{"grid":false},"sort":null,"type":"quantitative","field":"Humidity","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"tooltipDefaultMode":true}}""") if 'DeepnoteChart' in globals() else _dntk.DeepnoteChart(humidity_df, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"trail","color":"#2266D3","tooltip":true},"encoding":{"x":{"axis":{"grid":false},"sort":"ascending","type":"temporal","field":"Time","scale":{"type":"linear","zero":false}},"y":{"axis":{"grid":false},"sort":null,"type":"quantitative","field":"Humidity","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"tooltipDefaultMode":true}}"""))()

In [21]:
(lambda: DeepnoteChart(temperature_df, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"trail","color":"#2266D3","tooltip":true},"encoding":{"x":{"axis":{"grid":false},"sort":"ascending","type":"temporal","field":"Time","scale":{"type":"linear","zero":false}},"y":{"axis":{"grid":false},"sort":null,"type":"quantitative","field":"Temperature","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"tooltipDefaultMode":true}}""") if 'DeepnoteChart' in globals() else _dntk.DeepnoteChart(temperature_df, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"trail","color":"#2266D3","tooltip":true},"encoding":{"x":{"axis":{"grid":false},"sort":"ascending","type":"temporal","field":"Time","scale":{"type":"linear","zero":false}},"y":{"axis":{"grid":false},"sort":null,"type":"quantitative","field":"Temperature","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"tooltipDefaultMode":true}}"""))()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8a9d9526-dc21-42d6-ba37-8f708634743d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>